In [1]:
#load Data
import pandas as pd
#data2.csv has all the information
df=pd.read_csv("./data-ner.csv")
tags=df['tag'].tolist()[1:]
words=df['word'].tolist()[1:]
colors=["azul","rojo","negro","plata","dorado","oro"]
models=["versa","spark","civic","tsuru","fusion","camaro"]
brands=["nissan","ford","mitsubishi"]

In [2]:
def replace_wd(replace_w,stacktags,stackwords,replace_list):
    mayor_stacktags=["-","-","-","-"]
    mayor_stackwords=["-","-","-","-"]
    for element in replace_list:
        stackwords[stacktags.index(replace_w)]=element
        mayor_stacktags=mayor_stacktags+stacktags+["-","-","-","-"]
        mayor_stackwords=mayor_stackwords+stackwords+["-","-","-","-"]
    return mayor_stacktags,mayor_stackwords
    

In [3]:
endtags=[]
endwords=[]
stacktags=[]
stackwords=[]
for tag,word in zip(tags,words):
    if tag=='-':
        if len(stacktags)>0:
            if "MODELO" in stacktags:
                stacktags,stackwords=replace_wd("MODELO",stacktags,stackwords,models)
            if "MARCA" in stacktags:
                stacktags,stackwords=replace_wd("MARCA",stacktags,stackwords,brands)
            if "COLOR" in stacktags:
                stacktags,stackwords=replace_wd("COLOR",stacktags,stackwords,colors)
            endtags=endtags+stacktags
            endwords=endwords+stackwords
        stacktags=[]
        stackwords=[]
        endtags.append("-")
        endwords.append("-")
    else:
        stacktags.append(tag)
        stackwords.append(word)

In [4]:
dicts={}
dicts["tag"]=endtags
dicts["word"]=endwords
pd.DataFrame.from_dict(dicts)
df.to_csv("augmented.csv", sep='\t', encoding='utf-8')

In [5]:
def prepara_frase(tags,words):
    features=[]
    feature={}
    targets=[]
    for ind,tag in enumerate(tags):
        if tag!='-':
            feature['0']=words[ind-2]
            feature['1']=words[ind-1]
            feature['2']=words[ind]
            feature['3']=words[ind+1]
            feature['4']=words[ind+2]
            features.append(feature)
            #print feature
            feature={}
            #if vector[0]!='af':
            targets.append(tag)
            #else:
                #targets.append('*')
    return features,targets
features,targets=prepara_frase(endtags,endwords)
#Vectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.externals import joblib
vectorizer = DictVectorizer(sparse=False)
vectorizer.fit(features)
joblib.dump(vectorizer, 'vectorizer.pkl')
transformed=vectorizer.transform(features)
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(transformed, targets, test_size=0.1, random_state=42)
from sklearn import svm
lin_svc = svm.LinearSVC(C=1).fit(X_train, y_train)
#evaluando el modelo
from sklearn.metrics import accuracy_score
print(len(X_test))
print(accuracy_score(y_test, lin_svc.predict(X_test)))
joblib.dump(lin_svc, 'clasifier.pkl')


/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


36
1.0


['clasifier.pkl']